In [2]:
# 필요한 라이브러리 import하기

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import optuna 
import lightgbm as lgb 
from sklearn.metrics import accuracy_score, roc_auc_score,precision_score, confusion_matrix,recall_score 
from sklearn.model_selection import cross_val_score, train_test_split
import os 

/dshome/WoongLab/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
!nvidia-smi

Sat Apr 29 17:18:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.54       Driver Version: 510.54       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:03:00.0 Off |                  N/A |
|  0%   41C    P8    20W / 250W |      0MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:04:00.0 Off |                  N/A |
|  0%   

In [4]:
# train data와 test data 로드하기

train_data=pd.read_csv('/dshome/WoongLab/heo/construction_oil/preprocessed_data/pima_train_data.csv')
test_data=pd.read_csv('/dshome/WoongLab/heo/construction_oil/preprocessed_data/pima_test_data.csv')

In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               614 non-null    int64  
 1   Glucose                   614 non-null    int64  
 2   BloodPressure             614 non-null    int64  
 3   SkinThickness             614 non-null    int64  
 4   Insulin                   614 non-null    int64  
 5   BMI                       614 non-null    float64
 6   DiabetesPedigreeFunction  614 non-null    float64
 7   Age                       614 non-null    int64  
 8   Outcome                   614 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 43.3 KB


In [6]:
train_X=train_data.drop('Outcome',axis=1)
train_y=train_data['Outcome']

new_test_data_X=test_data.drop('Outcome',axis=1)
new_test_data_y=test_data['Outcome']

In [7]:
train_y.value_counts()

0    400
1    214
Name: Outcome, dtype: int64

In [8]:
# class weight 적용하기 위해 y label의 비율을 구하기

ratio=train_y.value_counts()[0]/train_y.value_counts()[1]

In [9]:
lgb=lgb.LGBMClassifier(learning_rate= 0.08674094387313179,max_depth=4,n_estimators=653,scale_pos_weight=ratio,random_state=42)

In [10]:
# 모델 학습하기

lgb.fit(train_X,train_y)

LGBMClassifier(learning_rate=0.08674094387313179, max_depth=4, n_estimators=653,
               random_state=42, scale_pos_weight=1.8691588785046729)

In [11]:
# 모델의 성능(AUC)을 평가하기 위해 예측된 클래스가 불량일 확률 구하기

prob=lgb.predict_proba(new_test_data_X)[:,1]

In [12]:
# 모델 성능 평가함수 만들기

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

def get_clf_prob(y_test, probability):
  pred=np.where(probability > 0.5,1,0)
  confusion=confusion_matrix(y_test, pred)
  accuracy=accuracy_score(y_test,pred)
  precision=precision_score(y_test,pred) 
  recall=recall_score(y_test,pred) 
  # F1 스코어 추가 
  f1=f1_score(y_test,pred,average='macro')
  Roc_score=roc_auc_score(y_test,probability)
  print('임계값: ', 0.5) 
  print('오차행렬')
  print(confusion) 
  # f1 score print 추가 
  print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1:{3:.4f}, AUC:{4: .4f}'.format(accuracy,precision,recall,f1,Roc_score))

In [13]:
# 모델 평가하기

get_clf_prob(new_test_data_y,prob)

임계값:  0.5
오차행렬
[[79 21]
 [19 35]]
정확도: 0.7403, 정밀도: 0.6250, 재현율: 0.6481, F1:0.7172, AUC: 0.7935
